## Objective: Transform Pipeline for model training
* VENV: abdomenct_seg

In [25]:
from pathlib import PosixPath, Path
import SimpleITK as sitk
import itkwidgets
import nibabel as nib
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import os
import numpy as np ## Has to be < 2.0 for monai to not break
import json
from typing import List
from monai import transforms as T

In [26]:
flag_env = load_dotenv( os.path.join("/".join(os.getcwd().split("/")[:-2]),  "envs/abdomenct_seg.env"))
if flag_env:
    root_dir = Path(os.environ.get("ROOT_DIR"))
else:
    raise Exception(f"Failed to load the environment file!")

In [28]:
save_dir = Path(os.environ.get("PROCESSED_DIR"))

In [29]:
img_prefix = "FLARE22"
img_type = "Tr"
img_id = "0001"
tmp_dict = {
    "image": str(root_dir / "imagesTr" / f"{img_prefix}_{img_type}_{img_id}_0000.nii.gz"),
    "label": str(root_dir / "labelsTr" / f"{img_prefix}_{img_type}_{img_id}.nii.gz"),
}

In [30]:
hyperparam_dict = {
    "data": {
        "original_max": -50,
        "original_min": 450,
        "scaled_max": 1.0,
        "scaled_min": 0.0,
    }
}

In [31]:
loader_transform = T.Compose([
    T.LoadImaged(
        keys=['image', 'label'],
        ensure_channel_first=True, ## Necessary for the other transforms
    ),
    T.ScaleIntensityRanged(
        keys=['image'],
        a_min=hyperparam_dict['data']['original_max'],
        a_max=hyperparam_dict['data']['original_min'],
        b_min=hyperparam_dict['data']['scaled_min'],
        b_max=hyperparam_dict['data']['scaled_max'],
        clip=True,
    ),
    T.CropForegroundd(keys=["image", "label"], source_key="image"),
    T.Orientationd(keys=["image", "label"], axcodes="RAS"),
    T.Spacingd(
        keys=["image", "label"],
        pixdim=(1.5, 1.5, 1.5),
        mode=("bilinear", "nearest"),
    ),
])

monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.


In [32]:
out_dict = loader_transform(tmp_dict)

In [34]:
processed_img = out_dict['image'].numpy()
processed_seg = out_dict['label'].numpy()

In [ ]:
flag_env = load_dotenv( os.path.join("/".join(os.getcwd().split("/")[:-2]),  "envs/abdomenct_seg.env"))
if flag_env:
    root_dir = Path(os.environ.get("ROOT_DIR"))
else:
    raise Exception(f"Failed to load the environment file!")

In [40]:
## Takes approximately 30-40 MiB. So the entire dataset is about ~3 GiB
np.savez(
    file=save_dir / "train" / f"{img_prefix}_preprocessed_img_{img_id}.npz", 
    arr=processed_img)
np.savez(
    file=save_dir / "train" / f"{img_prefix}_preprocessed_seg_{img_id}.npz", 
    arr=processed_img)

### Visualize Processed Volume & Seg:

In [20]:
itk_img = sitk.GetImageFromArray(out_dict['image'].squeeze().numpy())
itk_seg = sitk.GetImageFromArray(out_dict['label'].squeeze().numpy())

In [21]:
itkwidgets.Viewer(image=itk_img, label_image=itk_seg)

Viewer(interpolation=False, rendered_image=<itk.itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkI…